In [ ]:
import os
import pandas as pd
import numpy as np
import argparse
import ast
from together import Together
from alive_progress import alive_bar

from prompt import *
from classes.utils import extract_option

PATH = "/home/cc/PHD/HealthBranches/"

In [ ]:
df = pd.read_csv(f"{PATH}results/results_quiz_Llama-3.3-70B-Instruct-Turbo-Free.csv")
df["zero_shot_new"] = df["zero_shot"].apply(extract_option)

big_filtered = df[df["zero_shot_new"].str.upper() != df["real"].str.upper()]
len(big_filtered)

In [ ]:
small_df = pd.read_csv(f"{PATH}results/results_quiz_mistral.csv")
small_df = small_df[small_df["question"].isin(big_filtered["question"])]

In [ ]:
small_df["zero_shot_new"] = small_df["zero_shot"].apply(extract_option)
small_filtered = small_df[small_df["zero_shot_new"].str.upper() != small_df["real"].str.upper()]

In [ ]:
small_filtered